<span style="color:red"><b>Please note: you do not have to understand the content of this notebook yet. The rest of the course will walk you through the details. This introduction is here in case you are undecided which framework you would like to use and to serve as a reference material for operations that we will use frequently.</b></span>

# Getting started

[Cirq](https://github.com/quantumlib/Cirq) is Google's python framework for creating, editing, and invoking Noisy Intermediate Scale Quantum (NISQ) circuits. For setting it up on your computer, please refer to the [Cirq documentation](https://cirq.readthedocs.io/en/stable/). Here we spell out the details of Cirq that are critical for the rest of the notebooks.

The primary design philosophy behind Cirq is that the details of the quantum hardware can't be ignored for NISQ algorithms. Consequently, Cirq's abstractions closely mimic the underlying hardware. When declaring a qubit register, the connectivity structure must be specified. For our purposes, the most important basic elements of Cirq programs are (grid) qubits and circuits:

In [ ]:
import cirq
import numpy as np
from cirq import GridQubit, Circuit

Besides `GridQubit`, another possibility is `LineQubit`.

Conceptually, a `Circuit` object is very closely related to the abstract quantum circuit model. The quantum circuit takes gates that operate on the quantum registers. When a measurement gate is applied, the result is identified by a key. (In addition to `Circuit`, Cirq has another representation of quantum circuits called `Schedule`, which is more closely tied to hardware and includes detailed information about the timing and duration of the gates.)

Once we define our algorithm in terms of gates and measurements, we need to execute the circuit. In Cirq, the simulators make a distinction between a "run" and a "simulation". A "run" mimics the actual quantum hardware and does not allow access to the amplitudes of the wave function of the system. A "simulation" allows for operations which would not be possible on hardware, such as examining the wave function.

Cirq comes with a simulator for generic gates that implements their unitary matrix, and there is also another simulator which is customized for the native gate set of Google's Xmon hardware:

In [ ]:
from cirq import Simulator
from cirq.google import XmonSimulator

# Backends

The most straightforward simulator backend does exactly what we would expect: it runs a quantum algorithm and writes the measurement results out classically. After running a circuit a few times on the simulator, we can inspect the statistics of the results. For simplicity, we'll use the `Simulator` class:

In [ ]:
simulator = Simulator()

Let us build the simplest possible circuit that has no gates and only a measurement on a single qubit, writing out the result classically to the key `m`:

In [ ]:
q = GridQubit(0,0)
circuit = Circuit.from_ops(
    cirq.measure(q, key='m')
)

We execute this circuit on the simulator and observe the statistics:

In [ ]:
result = simulator.run(circuit, repetitions=100)
print(result.histogram(key='m'))

Remember that the qubit registers are always initialized as $|0\rangle$. Not surprisingly, out of a hundred executions, we measure `0` a hundred times. If you executed this on hardware, your measurement outcomes might be sometimes `1`  -- that would be due to noise.

If `run` was the only way to perform a simulation, we would have a hard time debugging our quantum algorithms. Why? As we don't have access to the quantum state using `run`, we would have to reconstruct the quantum state based on the measurements we make, which is not a trivial task in general. True, this is the only option we have on the actual hardware, but in a simulator, we have one more possibility: we could actually inspect the simulated quantum state (the wavefunction). Cirq provides a way to do this using the `simulate` method.

In this case, we do not have to add measurements, unless the protocol we are implementing uses a measurement in its internal operation. So we can build a circuit without a measurement and inspect the quantum state directly. In Cirq, it isn't possible to have an empty circuit.

In [ ]:
circuit = Circuit.from_ops(
    cirq.SingleQubitMatrixGate(np.array([[1, 0], [0, 1]])).on(q)
)
result = simulator.simulate(circuit)
print(result.final_state)

So in this case, we see it is the $|0\rangle$ state, as opposed to observing just the measurement statistics. This is especially important because the type of measurements we can perform are extremely restricted: technically speaking, we always measure in the computational basis. This means that, for instance, the states $|1\rangle$ and $-|1\rangle$ are indistinguishable based on the measurement statistics.

# Visualization

There are three handy ways of visualizing what we are doing. The first one is drawing the circuit. Cirq has built-in functionality to convert a circuit into a text diagram:

In [ ]:
q = GridQubit(0,0)
circuit = Circuit.from_ops(
    cirq.measure(q, key='m')
)
print(circuit.to_text_diagram())

This gives a quick sanity check to see whether we correctly implemented some circuit.

Cirq can also generate a LaTeX circuit diagram using the qcircuit package. This makes it easy to create our own function to plot the circuit graphically. This and the following helper functions will be included in `cirq_tools.py`, so we can just import it later.

In [ ]:
from cirq.contrib.qcircuit.qcircuit_diagram import circuit_to_latex_using_qcircuit
from pylatex import Document, NoEscape, Package
from tempfile import mkdtemp
import matplotlib.pyplot as plt
import shutil
import subprocess
%matplotlib inline

def plot_circuit(circuit):
    tex = circuit_to_latex_using_qcircuit(circuit)
    doc = Document(documentclass='standalone',
                   document_options=NoEscape('border=25pt,convert={density=300,outext=.png}'))
    doc.packages.append(Package('amsmath'))
    doc.packages.append(Package('qcircuit'))
    doc.append(NoEscape(tex))
    tmp_folder = mkdtemp()
    doc.generate_tex(tmp_folder + '/circuit')
    proc = subprocess.Popen(['pdflatex', '-shell-escape', tmp_folder + '/circuit.tex'], cwd=tmp_folder)
    proc.communicate()
    image = plt.imread(tmp_folder + '/circuit.png')
    shutil.rmtree(tmp_folder)
    plt.axis('off')
    return plt.imshow(image)

plot_circuit(circuit)

The second helper function shows the operation on the Bloch sphere, which is especially important for understanding how rotations happen. We borrowed this function from [this tutorial](https://github.com/markf94/rigetti_training_material) and it requires QuTiP. This visualization method relies on the wavefunction simulator.

In [ ]:
import cmath
from qutip import Bloch

def get_vector(alpha, beta):
    """
    Function to compute 3D Cartesian coordinates
    from 2D qubit vector.
    """

    # get phases
    angle_alpha = cmath.phase(alpha)
    angle_beta = cmath.phase(beta)

    # avoiding wrong normalization due to rounding errors
    if cmath.isclose(angle_alpha, cmath.pi):
        angle_alpha = 0
    if cmath.isclose(angle_beta, cmath.pi):
        angle_beta = 0
        
    if (angle_beta < 0 and angle_alpha < angle_beta) or (angle_beta > 0 and angle_alpha > angle_beta):
            denominator = cmath.exp(1j*angle_beta)
    else:
            denominator = cmath.exp(1j*angle_alpha)

    # eliminate global phase
    alpha_new = alpha/denominator
    beta_new = beta/denominator

    # special case to avoid division by zero
    if abs(alpha) == 0 or abs(beta) == 0:
        if alpha == 0:
            return [0,0,-1]
        else:
            return [0,0,1]
    else:
        # compute theta and phi from alpha and beta
        theta = 2*cmath.acos(alpha_new)
        phi = -1j*cmath.log(beta_new/cmath.sin(theta/2))

        # compute the Cartesian coordinates
        x = cmath.sin(theta)*cmath.cos(phi)
        y = cmath.sin(theta)*cmath.sin(phi)
        z = cmath.cos(theta)

    return [x.real, y.real, z.real]

def plot_quantum_state(amplitudes):
    """
    Thin function to abstract the plotting on the Bloch sphere.
    """
    bloch_sphere = Bloch()
    vec = get_vector(amplitudes[0], amplitudes[1])
    bloch_sphere.add_vectors(vec)
    bloch_sphere.show()
    bloch_sphere.clear()

 For instance, let's compare the initial state $|0\rangle$ and the Hadamard gate applied to it:

In [ ]:
circuit = Circuit.from_ops(
    cirq.measure(q, key='m')
)
result = simulator.simulate(circuit)
plot_quantum_state(result.final_state)

After the Hadamard gate:

In [ ]:
circuit = Circuit.from_ops(
    cirq.H(q),
    cirq.measure(q, key='m')
)
result = simulator.simulate(circuit)
print("After a Hadamard gate")
plot_quantum_state(result.final_state)

The third way of visualizing what happens is plotting the statistics of measurement results. Arguably, this is the most important for practical applications and debugging. We define a function for this:

In [ ]:
import matplotlib.pyplot as plt

def plot_histogram(counts):
    x = np.arange(len(counts))
    plt.bar(x, counts.values())
    plt.xticks(x, counts.keys())
    plt.show()

Here are the statistics before the Hadamard gate:

In [ ]:
circuit = Circuit.from_ops(
    cirq.measure(q, key='m')
)
results = simulator.run(circuit, repetitions=1000)
plot_histogram(results.histogram(key='m'))

After the Hadamard gate:

In [ ]:
circuit = Circuit.from_ops(
    cirq.H(q),
    cirq.measure(q, key='m')
)
results = simulator.run(circuit, repetitions=1000)
plot_histogram(results.histogram(key='m'))

As we can see, the 'perfect' nature of the simulator is reflected again in getting all 0s for the initial state, and a distribution very close to uniform after applying the Hadamard gate. In a longer circuit on real quantum hardware, these statistics would be heavily affected by noise.